<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#my_timeit" data-toc-modified-id="my_timeit-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>my_timeit</a></span></li><li><span><a href="#inner-vs.-dot-vs.-product-then-sum" data-toc-modified-id="inner-vs.-dot-vs.-product-then-sum-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><code>inner</code> vs. <code>dot</code> vs. product then <code>sum</code></a></span></li><li><span><a href="#dot-vs.-einsum" data-toc-modified-id="dot-vs.-einsum-3"><span class="toc-item-num">3&nbsp;&nbsp;</span><code>dot</code> vs. <code>einsum</code></a></span></li><li><span><a href="#einsum-vs.--product-then-sum" data-toc-modified-id="einsum-vs.--product-then-sum-4"><span class="toc-item-num">4&nbsp;&nbsp;</span><code>einsum</code> vs.  product then <code>sum</code></a></span></li></ul></div>

In [120]:
import numpy as np
import sys
print(sys.version)
print(sys.version_info)

3.6.7 |Anaconda, Inc.| (default, Dec 10 2018, 20:35:02) [MSC v.1915 64 bit (AMD64)]
sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)


## my_timeit

In [107]:
import timeit


def my_timeit(stat):
    timer = timeit.Timer(stat, globals={"np": np, "X": X, "x": x, "Y": Y})
    n, time = timer.autorange()
    time = time / n
    if time < 1E-3:
        print("%-30s [%.1f µs]" % (stat, time * 1E6))
    elif time < 1:
        print("%-30s [%.1f ms]" % (stat, time * 1E3))
    else:
        print("%-30s [%.1f s]" % (stat, time))

## `inner` vs. `dot` vs. product then `sum`

In [108]:
N = 1000
x = np.random.randn(N)
my_timeit("np.dot(x,x)")
my_timeit("np.inner(x,x)")
my_timeit("np.linalg.norm(x, ord=2)")
my_timeit("np.linalg.norm(x, ord=2)**2")

np.dot(x,x)                    [1.1 µs]
np.inner(x,x)                  [1.2 µs]
np.linalg.norm(x, ord=2)       [4.9 µs]
np.linalg.norm(x, ord=2)**2    [5.4 µs]


## `dot` vs. `einsum`

In [118]:
N = 1000
M = 999
X = np.random.randn(N, M)
Y = np.random.randn(N, M)
assert np.allclose(X.dot(Y.T), np.einsum("ij,kj->ik", X, Y))

my_timeit("np.sum(X*Y, axis=0)")
my_timeit('np.einsum("ij,ij->j",X, Y)')

np.sum(X*Y, axis=0)            [3.9 ms]
np.einsum("ij,ij->j",X, Y)     [1.0 ms]


## `einsum` vs.  product then `sum` 

In [114]:
N = 1000
M = 999
X = np.random.randn(N, M)
Y = np.random.randn(N, M)
assert np.allclose(np.sum(X * Y, axis=0), np.einsum("ij,ij->j", X, Y))

my_timeit("np.sum(X*Y, axis=0)")
my_timeit('np.einsum("ij,ij->j",X, Y)')

np.sum(X*Y, axis=0)            [3.7 ms]
np.einsum("ij,ij->j",X, Y)     [976.3 µs]
